In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [46]:
from function import data_loader
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

In [67]:
dfs = data_loader(path='../data/', city='London')

In [68]:
df1 = dfs['london_tickets_for_sale']
df2 = dfs['london_ticket_sales']
df2.rename(columns={'session': 'code'}, inplace=True)

session_map = df1[['code', 'venue', 'event']].drop_duplicates().set_index('code')

# Join the two data frames

Not that easy, information is more granular on 'london_tickets_for_sale', and 'london_ticket_sales' covers much less events.

In [83]:
# Hacky but that'll do

def venue_lookup(code):
    try:
        venue = session_map.at[code, 'venue']
        if 'Olympic Park' in venue:
            return 'Olympic Park'
        else:
            return venue
    except:
        return None

def event_lookup(code):
    try:
        return session_map.at[code, 'event']
    except:
        return None

    
df2['venue'] = df2.apply(lambda row: venue_lookup(row.code), axis=1)
df2['event'] = df2.apply(lambda row: event_lookup(row.code), axis=1)

In [87]:
df1[df1['venue'] == 'Hyde Park']

,date,time,gender,session,medal_ceremony,code,AA,A,B,C,D,E,limit,venue,event
709,09-Aug,12:0015:00,W,Marathon Swimming 10km; victory ceremony,Y,SM001,,60,40,20,,,10.0,Hyde Park,Swimming - Marathon
710,10-Aug,12:0015:00,M,Marathon Swimming 10km; victory ceremony,Y,SM002,,60,40,20,,,10.0,Hyde Park,Swimming - Marathon
776,04-Aug,09:0011:40,W,Triathlon; victory ceremony,Y,TR001,,60,40,20,,,10.0,Hyde Park,Triathlon
777,07-Aug,11:3014:00,M,Triathlon; victory ceremony,Y,TR002,,60,40,20,,,10.0,Hyde Park,Triathlon


In [88]:
df2[df2['venue'] == 'Hyde Park']

,code,price,number_sold,total_public_sold,percent_sold_to_public,venue,event
892,TR001,60,314,1248,29.111267,Hyde Park,Triathlon
893,TR001,40,111,1248,29.111267,Hyde Park,Triathlon
894,TR001,20,642,1248,29.111267,Hyde Park,Triathlon
895,TR001,Special,181,1248,29.111267,Hyde Park,Triathlon
896,TR002,60,240,1046,24.179380,Hyde Park,Triathlon
897,TR002,40,93,1046,24.179380,Hyde Park,Triathlon
898,TR002,20,582,1046,24.179380,Hyde Park,Triathlon
899,TR002,Special,131,1046,24.179380,Hyde Park,Triathlon


# Total number of tickets sold to public by venues

Is it usable? That does not look too good:
1) it is only Central London (although that is the most informational I guess)
2) some numbers are really low (Hyde Park : 2294 tickets sold? that's like 30s of traffic in a regular day)
3) poor coverage

In [90]:
df2.groupby('venue')['number_sold'].sum()

venue
ExCeL                     162677
Greenwich Park            205899
Hyde Park                   2294
North Greenwich Arena     122524
Olympic Park             1374980
The Mall                    3867
Wembley Arena              10746
Name: number_sold, dtype: int64